In [1]:
import pandas as pd
import duckdb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting and display options
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 50)

print("🔍 POSITION-SPECIFIC TECHNICAL ANALYSIS")
print("="*60)
print("Analyzing individual player performance by role using comprehensive stats")

# Connect to database
db_path = "../data/premierleague_raw.duckdb"
conn = duckdb.connect(db_path)

# Define Big 6 teams
BIG_6 = ['Manchester City', 'Liverpool', 'Arsenal', 'Chelsea', 'Manchester Utd', 'Tottenham']

print("\n🎯 Analyzing Big 6 technical performance by position:")
for i, team in enumerate(BIG_6, 1):
    print(f"  {i}. {team}")


🔍 POSITION-SPECIFIC TECHNICAL ANALYSIS
Analyzing individual player performance by role using comprehensive stats

🎯 Analyzing Big 6 technical performance by position:
  1. Manchester City
  2. Liverpool
  3. Arsenal
  4. Chelsea
  5. Manchester Utd
  6. Tottenham


In [2]:
print("\n⚽ FORWARD ANALYSIS - Big 6 Attackers Technical Breakdown")
print("-"*60)

big6_forwards = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    ps."Playing Time 90s" as Games_90s,
    
    -- ATTACKING OUTPUT
    ROUND(CAST(ps."Performance Gls" as FLOAT) / ps."Playing Time 90s", 2) as Goals_per_90,
    ROUND(CAST(ps."Performance Ast" as FLOAT) / ps."Playing Time 90s", 2) as Assists_per_90,
    ROUND(CAST(ps."Performance G+A" as FLOAT) / ps."Playing Time 90s", 2) as G_A_per_90,
    
    -- SHOOTING EFFICIENCY
    ROUND(CAST(psh."Standard Sh" as FLOAT) / psh."90s", 2) as Shots_per_90,
    psh."Standard SoT%" as Shot_Accuracy,
    psh."Standard G/SoT" as Goals_per_SoT,
    psh."Standard Dist" as Avg_Shot_Distance,
    
    -- EXPECTED GOALS
    ROUND(CAST(psh."Expected xG" as FLOAT) / psh."90s", 2) as xG_per_90,
    psh."Expected G-xG" as Goals_vs_xG,
    ROUND(CAST(psh."Expected npxG" as FLOAT) / psh."90s", 2) as npxG_per_90,
    
    -- CHANCE CREATION
    ROUND(CAST(pp.KP as FLOAT) / pp."90s", 2) as Key_Passes_per_90,
    ROUND(CAST(pp.xAG as FLOAT) / pp."90s", 2) as xAG_per_90,
    
    -- MOVEMENT & PROGRESSION
    ROUND(CAST(po."Touches Att Pen" as FLOAT) / po."90s", 2) as Pen_Box_Touches_per_90,
    ROUND(CAST(po."Take-Ons Succ" as FLOAT) / po."90s", 2) as Successful_Takeons_per_90,
    po."Take-Ons Succ%" as Takeon_Success_Rate,
    
    -- WORK RATE
    ROUND(CAST(pm."Performance Recov" as FLOAT) / pm."90s", 2) as Ball_Recoveries_per_90,
    ROUND(CAST(pm."Performance Fld" as FLOAT) / pm."90s", 2) as Fouls_Drawn_per_90

FROM player_standard ps
LEFT JOIN player_shooting psh ON ps.Player = psh.Player
LEFT JOIN player_passing pp ON ps.Player = pp.Player  
LEFT JOIN player_possession po ON ps.Player = po.Player
LEFT JOIN player_misc pm ON ps.Player = pm.Player
WHERE 
    ps.Squad IN ('Manchester City', 'Liverpool', 'Arsenal', 'Chelsea', 'Manchester Utd', 'Tottenham')
    AND ps.Pos LIKE '%FW%'
    AND ps."Playing Time 90s" >= 2
ORDER BY G_A_per_90 DESC
""").fetchdf()

print("🥅 BIG 6 FORWARDS - ATTACKING OUTPUT:")
print(big6_forwards[['Player', 'Squad', 'Goals_per_90', 'Assists_per_90', 'G_A_per_90', 'Games_90s']].head(12))

print("\n🎯 BIG 6 FORWARDS - SHOOTING EFFICIENCY:")
shooting_efficiency = big6_forwards.nlargest(10, 'Goals_per_SoT')
print(shooting_efficiency[['Player', 'Squad', 'Shot_Accuracy', 'Goals_per_SoT', 'Shots_per_90', 'Avg_Shot_Distance']])

print("\n⚡ BIG 6 FORWARDS - FINISHING QUALITY (Goals vs xG):")
finishing_quality = big6_forwards.nlargest(8, 'Goals_vs_xG') 
print(finishing_quality[['Player', 'Squad', 'Goals_vs_xG', 'xG_per_90', 'npxG_per_90']])

print("\n🎨 BIG 6 FORWARDS - CREATIVITY & CHANCE CREATION:")
creative_forwards = big6_forwards.nlargest(10, 'Key_Passes_per_90')
print(creative_forwards[['Player', 'Squad', 'Key_Passes_per_90', 'xAG_per_90', 'Assists_per_90']])

print("\n🏃 BIG 6 FORWARDS - MOVEMENT & DRIBBLING:")
movement_forwards = big6_forwards.nlargest(8, 'Successful_Takeons_per_90')
print(movement_forwards[['Player', 'Squad', 'Pen_Box_Touches_per_90', 'Successful_Takeons_per_90', 'Takeon_Success_Rate']])

# Team comparison
print("\n📊 FORWARD PERFORMANCE BY TEAM:")
forward_team_stats = big6_forwards.groupby('Squad').agg({
    'Goals_per_90': 'mean',
    'Assists_per_90': 'mean', 
    'Shot_Accuracy': 'mean',
    'Key_Passes_per_90': 'mean',
    'Successful_Takeons_per_90': 'mean'
}).round(2).sort_values('Goals_per_90', ascending=False)

for team, stats in forward_team_stats.iterrows():
    print(f"  {team}:")
    print(f"    Goals per 90: {stats['Goals_per_90']:.2f} | Shot accuracy: {stats['Shot_Accuracy']:.1f}%")
    print(f"    Key passes per 90: {stats['Key_Passes_per_90']:.2f} | Takeons per 90: {stats['Successful_Takeons_per_90']:.2f}")



⚽ FORWARD ANALYSIS - Big 6 Attackers Technical Breakdown
------------------------------------------------------------
🥅 BIG 6 FORWARDS - ATTACKING OUTPUT:
             Player            Squad  Goals_per_90  Assists_per_90  \
0        João Pedro          Chelsea          0.71            0.71   
1      Hugo Ekitike        Liverpool          0.80            0.40   
2       Richarlison        Tottenham          0.77            0.38   
3    Erling Haaland  Manchester City          1.07            0.00   
4        Cody Gakpo        Liverpool          0.33            0.67   
5   Brennan Johnson        Tottenham          0.87            0.00   
6   Viktor Gyökeres          Arsenal          0.74            0.00   
7    Mohammed Kudus        Tottenham          0.00            0.69   
8     Mohamed Salah        Liverpool          0.33            0.33   
9   Estêvão Willian          Chelsea          0.00            0.50   
10       Oscar Bobb  Manchester City          0.00            0.36   
11  

In [3]:
print("\n🎯 MIDFIELDER ANALYSIS - Big 6 Midfielders Technical Breakdown")
print("-"*60)

big6_midfielders = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    ps."Playing Time 90s" as Games_90s,
    
    -- PASSING QUALITY
    pp."Total Cmp%" as Pass_Accuracy,
    ROUND(CAST(pp."Total Att" as FLOAT) / pp."90s", 2) as Pass_Attempts_per_90,
    ROUND(CAST(pp.PrgP as FLOAT) / pp."90s", 2) as Progressive_Passes_per_90,
    ROUND(CAST(pp."1/3" as FLOAT) / pp."90s", 2) as Final_Third_Passes_per_90,
    
    -- CREATIVITY
    ROUND(CAST(pp.KP as FLOAT) / pp."90s", 2) as Key_Passes_per_90,
    ROUND(CAST(pp.xAG as FLOAT) / pp."90s", 2) as xAG_per_90,
    ROUND(CAST(ps."Performance Ast" as FLOAT) / ps."Playing Time 90s", 2) as Assists_per_90,
    ROUND(CAST(pg."SCA SCA" as FLOAT) / pg."90s", 2) as Shot_Creating_Actions_per_90,
    
    -- BALL PROGRESSION  
    ROUND(CAST(ps."Progression PrgC" as FLOAT) / ps."Playing Time 90s", 2) as Progressive_Carries_per_90,
    ROUND(CAST(po."Carries 1/3" as FLOAT) / po."90s", 2) as Final_Third_Carries_per_90,
    ROUND(CAST(po."Touches Touches" as FLOAT) / po."90s", 2) as Touches_per_90,
    
    -- DEFENSIVE CONTRIBUTION
    ROUND(CAST(pd."Tackles Tkl" as FLOAT) / pd."90s", 2) as Tackles_per_90,
    ROUND(CAST(pd.Int as FLOAT) / pd."90s", 2) as Interceptions_per_90,
    ROUND(CAST(pm."Performance Recov" as FLOAT) / pm."90s", 2) as Ball_Recoveries_per_90,
    pd."Challenges Tkl%" as Tackle_Success_Rate,
    
    -- ATTACKING THREAT
    ROUND(CAST(ps."Performance Gls" as FLOAT) / ps."Playing Time 90s", 2) as Goals_per_90,
    ROUND(CAST(psh."Standard Sh" as FLOAT) / psh."90s", 2) as Shots_per_90,
    ROUND(CAST(psh."Expected xG" as FLOAT) / psh."90s", 2) as xG_per_90

FROM player_standard ps
LEFT JOIN player_shooting psh ON ps.Player = psh.Player
LEFT JOIN player_passing pp ON ps.Player = pp.Player  
LEFT JOIN player_defense pd ON ps.Player = pd.Player
LEFT JOIN player_possession po ON ps.Player = po.Player
LEFT JOIN player_misc pm ON ps.Player = pm.Player
LEFT JOIN player_goalshotcreation pg ON ps.Player = pg.Player
WHERE 
    ps.Squad IN ('Manchester City', 'Liverpool', 'Arsenal', 'Chelsea', 'Manchester Utd', 'Tottenham')
    AND ps.Pos LIKE '%MF%'
    AND ps."Playing Time 90s" >= 2
ORDER BY Key_Passes_per_90 DESC
""").fetchdf()

print("🎯 BIG 6 MIDFIELDERS - CREATIVITY & PLAYMAKING:")
creative_mids = big6_midfielders.head(12)
print(creative_mids[['Player', 'Squad', 'Key_Passes_per_90', 'xAG_per_90', 'Assists_per_90', 'Shot_Creating_Actions_per_90']])

print("\n⚽ BIG 6 MIDFIELDERS - PASSING QUALITY & VOLUME:")
passing_mids = big6_midfielders.nlargest(10, 'Progressive_Passes_per_90')
print(passing_mids[['Player', 'Squad', 'Pass_Accuracy', 'Pass_Attempts_per_90', 'Progressive_Passes_per_90', 'Final_Third_Passes_per_90']])

print("\n🛡️ BIG 6 MIDFIELDERS - DEFENSIVE CONTRIBUTION:")
defensive_mids = big6_midfielders.nlargest(10, 'Tackles_per_90')
print(defensive_mids[['Player', 'Squad', 'Tackles_per_90', 'Tackle_Success_Rate', 'Interceptions_per_90', 'Ball_Recoveries_per_90']])

print("\n⚡ BIG 6 MIDFIELDERS - BOX-TO-BOX THREAT:")
box_to_box = big6_midfielders[big6_midfielders['Goals_per_90'] > 0].nlargest(8, 'Goals_per_90')
print(box_to_box[['Player', 'Squad', 'Goals_per_90', 'Shots_per_90', 'xG_per_90', 'Key_Passes_per_90']])

print("\n🏃 BIG 6 MIDFIELDERS - BALL PROGRESSION:")
progression_mids = big6_midfielders.nlargest(10, 'Progressive_Carries_per_90') 
print(progression_mids[['Player', 'Squad', 'Progressive_Carries_per_90', 'Final_Third_Carries_per_90', 'Touches_per_90']])

# Team comparison
print("\n📊 MIDFIELDER PERFORMANCE BY TEAM:")
mid_team_stats = big6_midfielders.groupby('Squad').agg({
    'Pass_Accuracy': 'mean',
    'Key_Passes_per_90': 'mean',
    'Progressive_Passes_per_90': 'mean', 
    'Tackles_per_90': 'mean',
    'Goals_per_90': 'mean'
}).round(2).sort_values('Key_Passes_per_90', ascending=False)

for team, stats in mid_team_stats.iterrows():
    print(f"  {team}:")
    print(f"    Pass accuracy: {stats['Pass_Accuracy']:.1f}% | Key passes: {stats['Key_Passes_per_90']:.2f}")
    print(f"    Progressive passes: {stats['Progressive_Passes_per_90']:.2f} | Tackles: {stats['Tackles_per_90']:.2f}")



🎯 MIDFIELDER ANALYSIS - Big 6 Midfielders Technical Breakdown
------------------------------------------------------------
🎯 BIG 6 MIDFIELDERS - CREATIVITY & PLAYMAKING:
                Player            Squad  Key_Passes_per_90  xAG_per_90  \
0      Bruno Fernandes   Manchester Utd               4.00        0.23   
1         Bryan Mbeumo   Manchester Utd               3.00        0.43   
2          Mason Mount   Manchester Utd               2.50        0.20   
3           Oscar Bobb  Manchester City               1.43        0.07   
4        Florian Wirtz        Liverpool               1.43        0.14   
5       Enzo Fernández          Chelsea               1.38        0.24   
6          Declan Rice          Arsenal               1.38        0.17   
7        Matheus Cunha   Manchester Utd               1.30        0.09   
8           João Pedro          Chelsea               1.07        0.36   
9    Tijjani Reijnders  Manchester City               1.00        0.10   
10  Dominik Szo

In [4]:
print("\n🛡️ DEFENDER ANALYSIS - Big 6 Defenders Technical Breakdown")
print("-"*60)

big6_defenders = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    ps."Playing Time 90s" as Games_90s,
    
    -- DEFENSIVE ACTIONS
    ROUND(CAST(pd."Tackles Tkl" as FLOAT) / pd."90s", 2) as Tackles_per_90,
    ROUND(CAST(pd."Tackles TklW" as FLOAT) / pd."90s", 2) as Tackles_Won_per_90,
    pd."Challenges Tkl%" as Tackle_Success_Rate,
    ROUND(CAST(pd.Int as FLOAT) / pd."90s", 2) as Interceptions_per_90,
    ROUND(CAST(pd.Clr as FLOAT) / pd."90s", 2) as Clearances_per_90,
    ROUND(CAST(pd."Blocks Blocks" as FLOAT) / pd."90s", 2) as Blocks_per_90,
    
    -- AERIAL ABILITY
    ROUND(CAST(pm."Aerial Duels Won" as FLOAT) / pm."90s", 2) as Aerials_Won_per_90,
    pm."Aerial Duels Won%" as Aerial_Win_Rate,
    
    -- BALL-PLAYING ABILITY
    pp."Total Cmp%" as Pass_Accuracy,
    ROUND(CAST(pp."Total Att" as FLOAT) / pp."90s", 2) as Pass_Attempts_per_90,
    pp."Long Cmp%" as Long_Pass_Accuracy,
    ROUND(CAST(pp.PrgP as FLOAT) / pp."90s", 2) as Progressive_Passes_per_90,
    
    -- BALL PROGRESSION
    ROUND(CAST(ps."Progression PrgC" as FLOAT) / ps."Playing Time 90s", 2) as Progressive_Carries_per_90,
    ROUND(CAST(po."Carries 1/3" as FLOAT) / po."90s", 2) as Final_Third_Carries_per_90,
    ROUND(CAST(po."Touches Touches" as FLOAT) / po."90s", 2) as Touches_per_90,
    
    -- ATTACKING CONTRIBUTION
    ROUND(CAST(ps."Performance Gls" as FLOAT) / ps."Playing Time 90s", 2) as Goals_per_90,
    ROUND(CAST(ps."Performance Ast" as FLOAT) / ps."Playing Time 90s", 2) as Assists_per_90,
    ROUND(CAST(pp.KP as FLOAT) / pp."90s", 2) as Key_Passes_per_90,
    
    -- DISCIPLINE
    ROUND(CAST(pm."Performance CrdY" as FLOAT) / pm."90s", 2) as Yellow_Cards_per_90,
    pd.Err as Errors_Leading_to_Shots,
    ROUND(CAST(pm."Performance Recov" as FLOAT) / pm."90s", 2) as Ball_Recoveries_per_90

FROM player_standard ps
LEFT JOIN player_shooting psh ON ps.Player = psh.Player
LEFT JOIN player_passing pp ON ps.Player = pp.Player  
LEFT JOIN player_defense pd ON ps.Player = pd.Player
LEFT JOIN player_possession po ON ps.Player = po.Player
LEFT JOIN player_misc pm ON ps.Player = pm.Player
WHERE 
    ps.Squad IN ('Manchester City', 'Liverpool', 'Arsenal', 'Chelsea', 'Manchester Utd', 'Tottenham')
    AND ps.Pos LIKE '%DF%'
    AND ps."Playing Time 90s" >= 2
ORDER BY Tackles_Won_per_90 DESC
""").fetchdf()

print("🛡️ BIG 6 DEFENDERS - DEFENSIVE ACTIONS:")
defensive_actions = big6_defenders.head(12)
print(defensive_actions[['Player', 'Squad', 'Tackles_per_90', 'Tackles_Won_per_90', 'Tackle_Success_Rate', 'Interceptions_per_90', 'Clearances_per_90']])

print("\n✈️ BIG 6 DEFENDERS - AERIAL DOMINANCE:")
aerial_defenders = big6_defenders.nlargest(10, 'Aerials_Won_per_90')
print(aerial_defenders[['Player', 'Squad', 'Aerials_Won_per_90', 'Aerial_Win_Rate', 'Clearances_per_90']])

print("\n⚽ BIG 6 DEFENDERS - BALL-PLAYING ABILITY:")
ball_playing_defenders = big6_defenders.nlargest(10, 'Progressive_Passes_per_90')
print(ball_playing_defenders[['Player', 'Squad', 'Pass_Accuracy', 'Long_Pass_Accuracy', 'Progressive_Passes_per_90', 'Pass_Attempts_per_90']])

print("\n🏃 BIG 6 DEFENDERS - BALL PROGRESSION:")
progressive_defenders = big6_defenders.nlargest(8, 'Progressive_Carries_per_90')
print(progressive_defenders[['Player', 'Squad', 'Progressive_Carries_per_90', 'Final_Third_Carries_per_90', 'Touches_per_90']])

print("\n⚡ BIG 6 DEFENDERS - ATTACKING THREAT:")
attacking_defenders = big6_defenders[big6_defenders['Goals_per_90'] > 0].nlargest(8, 'Goals_per_90')
if len(attacking_defenders) > 0:
    print(attacking_defenders[['Player', 'Squad', 'Goals_per_90', 'Assists_per_90', 'Key_Passes_per_90']])
else:
    print("No defenders with goals in current sample")

print("\n⚠️ BIG 6 DEFENDERS - DISCIPLINE & ERRORS:")
discipline_check = big6_defenders.nsmallest(8, 'Yellow_Cards_per_90')
print(discipline_check[['Player', 'Squad', 'Yellow_Cards_per_90', 'Errors_Leading_to_Shots', 'Ball_Recoveries_per_90']])

# Team comparison
print("\n📊 DEFENDER PERFORMANCE BY TEAM:")
def_team_stats = big6_defenders.groupby('Squad').agg({
    'Tackles_Won_per_90': 'mean',
    'Interceptions_per_90': 'mean',
    'Pass_Accuracy': 'mean',
    'Progressive_Passes_per_90': 'mean',
    'Aerial_Win_Rate': 'mean'
}).round(2).sort_values('Tackles_Won_per_90', ascending=False)

for team, stats in def_team_stats.iterrows():
    print(f"  {team}:")
    print(f"    Tackles won: {stats['Tackles_Won_per_90']:.2f} | Interceptions: {stats['Interceptions_per_90']:.2f}")
    print(f"    Pass accuracy: {stats['Pass_Accuracy']:.1f}% | Progressive passes: {stats['Progressive_Passes_per_90']:.2f}")



🛡️ DEFENDER ANALYSIS - Big 6 Defenders Technical Breakdown
------------------------------------------------------------
🛡️ BIG 6 DEFENDERS - DEFENSIVE ACTIONS:
                Player            Squad  Tackles_per_90  Tackles_Won_per_90  \
0      Rayan Aït-Nouri  Manchester City            4.09                2.73   
1   Dominik Szoboszlai        Liverpool            2.33                2.33   
2         Milos Kerkez        Liverpool            2.22                1.85   
3      Trevoh Chalobah          Chelsea            2.50                1.79   
4          Amad Diallo   Manchester Utd            3.00                1.50   
5        Patrick Dorgu   Manchester Utd            2.00                1.50   
6            Luke Shaw   Manchester Utd            2.50                1.43   
7       Marc Cucurella          Chelsea            2.14                1.07   
8          Pedro Porro        Tottenham            1.79                1.07   
9            Leny Yoro   Manchester Utd          

In [5]:
print("\n🥅 GOALKEEPER ANALYSIS - Big 6 Goalkeepers Technical Breakdown")
print("-"*60)

big6_goalkeepers = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps."Playing Time 90s" as Games_90s,
    
    -- SHOT STOPPING
    pk."Performance Save%" as Save_Percentage,
    pk."Performance GA90" as Goals_Against_per_90,
    ROUND(CAST(pk."Performance SoTA" as FLOAT) / pk."Playing Time 90s", 2) as Shots_Faced_per_90,
    pk."Performance CS%" as Clean_Sheet_Percentage,
    
    -- ADVANCED METRICS
    pka."Expected PSxG/SoT" as PSxG_per_Shot,
    pka."Expected PSxG+/-" as Goals_Prevented,
    pka."Expected /90" as Goals_Prevented_per_90,
    
    -- DISTRIBUTION
    pka."Launched Cmp%" as Long_Pass_Accuracy,
    ROUND(CAST(pka."Launched Att" as FLOAT) / pka."90s", 2) as Long_Passes_per_90,
    pka."Passes AvgLen" as Avg_Pass_Length,
    pka."Goal Kicks Launch%" as Goal_Kick_Launch_Rate,
    
    -- COMMAND & SWEEPING
    pka."Crosses Stp%" as Cross_Stop_Rate,
    ROUND(CAST(pka."Crosses Opp" as FLOAT) / pka."90s", 2) as Crosses_Faced_per_90,
    pka."Sweeper #OPA/90" as Sweeper_Actions_per_90,
    pka."Sweeper AvgDist" as Avg_Sweeper_Distance,
    
    -- PENALTY STOPPING
    pk."Penalty Kicks Save%" as Penalty_Save_Rate,
    pk."Penalty Kicks PKA" as Penalties_Faced

FROM player_standard ps
LEFT JOIN player_keepers pk ON ps.Player = pk.Player
LEFT JOIN player_keepersadv pka ON ps.Player = pka.Player
WHERE 
    ps.Squad IN ('Manchester City', 'Liverpool', 'Arsenal', 'Chelsea', 'Manchester Utd', 'Tottenham')
    AND ps.Pos LIKE '%GK%'
    AND ps."Playing Time 90s" >= 2
ORDER BY Save_Percentage DESC
""").fetchdf()

if len(big6_goalkeepers) > 0:
    print("🥅 BIG 6 GOALKEEPERS - SHOT STOPPING:")
    print(big6_goalkeepers[['Player', 'Squad', 'Save_Percentage', 'Goals_Against_per_90', 'Shots_Faced_per_90', 'Clean_Sheet_Percentage']])
    
    print("\n📈 BIG 6 GOALKEEPERS - ADVANCED METRICS:")
    print(big6_goalkeepers[['Player', 'Squad', 'PSxG_per_Shot', 'Goals_Prevented', 'Goals_Prevented_per_90']])
    
    print("\n⚽ BIG 6 GOALKEEPERS - DISTRIBUTION:")
    print(big6_goalkeepers[['Player', 'Squad', 'Long_Pass_Accuracy', 'Long_Passes_per_90', 'Avg_Pass_Length', 'Goal_Kick_Launch_Rate']])
    
    print("\n🛡️ BIG 6 GOALKEEPERS - COMMAND & SWEEPING:")
    print(big6_goalkeepers[['Player', 'Squad', 'Cross_Stop_Rate', 'Crosses_Faced_per_90', 'Sweeper_Actions_per_90']])
else:
    print("No goalkeeper data available in current dataset")



🥅 GOALKEEPER ANALYSIS - Big 6 Goalkeepers Technical Breakdown
------------------------------------------------------------
🥅 BIG 6 GOALKEEPERS - SHOT STOPPING:
              Player            Squad  Save_Percentage  Goals_Against_per_90  \
0  Guglielmo Vicario        Tottenham             92.9                  0.33   
1         David Raya          Arsenal             90.9                  0.33   
2     Robert Sánchez          Chelsea             88.9                  0.33   
3     James Trafford  Manchester City             80.0                  1.33   
4     Altay Bayındır   Manchester Utd             55.6                  1.33   
5            Alisson        Liverpool             42.9                  1.33   

   Shots_Faced_per_90  Clean_Sheet_Percentage  
0                4.67                    66.7  
1                3.67                    66.7  
2                3.00                    66.7  
3                5.00                    33.3  
4                3.00                 

In [6]:
print("\n📊 CROSS-POSITION ANALYSIS & TEAM SUMMARIES")
print("-"*60)

# Most complete players (high performance across multiple areas)
print("🌟 MOST COMPLETE PLAYERS BY POSITION:")

print("\n🥅 Complete Forwards (Output + Creativity + Work Rate):")
complete_forwards = big6_forwards[
    (big6_forwards['G_A_per_90'] >= 0.5) & 
    (big6_forwards['Key_Passes_per_90'] >= 1.0) &
    (big6_forwards['Ball_Recoveries_per_90'] >= 3.0)
].head(6)
if len(complete_forwards) > 0:
    print(complete_forwards[['Player', 'Squad', 'G_A_per_90', 'Key_Passes_per_90', 'Ball_Recoveries_per_90']])

print("\n🎯 Complete Midfielders (Creativity + Passing + Defense):")
complete_midfielders = big6_midfielders[
    (big6_midfielders['Key_Passes_per_90'] >= 1.5) &
    (big6_midfielders['Pass_Accuracy'] >= 85) &
    (big6_midfielders['Tackles_per_90'] >= 1.5)
].head(6)
if len(complete_midfielders) > 0:
    print(complete_midfielders[['Player', 'Squad', 'Key_Passes_per_90', 'Pass_Accuracy', 'Tackles_per_90']])

print("\n🛡️ Complete Defenders (Defense + Ball-Playing + Progression):")
complete_defenders = big6_defenders[
    (big6_defenders['Tackles_Won_per_90'] >= 1.0) &
    (big6_defenders['Pass_Accuracy'] >= 85) &
    (big6_defenders['Progressive_Passes_per_90'] >= 3.0)
].head(6)
if len(complete_defenders) > 0:
    print(complete_defenders[['Player', 'Squad', 'Tackles_Won_per_90', 'Pass_Accuracy', 'Progressive_Passes_per_90']])

# Team technical profiles
print("\n📈 TEAM TECHNICAL PROFILES:")
print("Comparing how each Big 6 team's players perform in key technical areas\n")

team_profiles = {}

# Forward profiles
if len(big6_forwards) > 0:
    fw_profiles = big6_forwards.groupby('Squad').agg({
        'Goals_per_90': 'mean',
        'Shot_Accuracy': 'mean',
        'Key_Passes_per_90': 'mean',
        'Successful_Takeons_per_90': 'mean'
    }).round(2)
    team_profiles['Forwards'] = fw_profiles

# Midfielder profiles  
if len(big6_midfielders) > 0:
    mid_profiles = big6_midfielders.groupby('Squad').agg({
        'Pass_Accuracy': 'mean',
        'Key_Passes_per_90': 'mean',
        'Tackles_per_90': 'mean',
        'Progressive_Passes_per_90': 'mean'
    }).round(2)
    team_profiles['Midfielders'] = mid_profiles

# Defender profiles
if len(big6_defenders) > 0:
    def_profiles = big6_defenders.groupby('Squad').agg({
        'Tackles_Won_per_90': 'mean',
        'Pass_Accuracy': 'mean',
        'Progressive_Passes_per_90': 'mean',
        'Aerial_Win_Rate': 'mean'
    }).round(2)
    team_profiles['Defenders'] = def_profiles

for position, profile_data in team_profiles.items():
    print(f"{position} Technical Profile by Team:")
    print(profile_data)
    print()

print("="*60)
print("🎯 ANALYSIS COMPLETE")
print("="*60)
print("Key Insights:")
print("• Individual player technical performance broken down by position")
print("• Role-specific metrics show strengths/weaknesses within positions") 
print("• Team profiles reveal tactical patterns and squad construction")
print("• Cross-position analysis identifies most complete players")
print("• Data enables informed player comparisons and tactical decisions")


📊 CROSS-POSITION ANALYSIS & TEAM SUMMARIES
------------------------------------------------------------
🌟 MOST COMPLETE PLAYERS BY POSITION:

🥅 Complete Forwards (Output + Creativity + Work Rate):
            Player      Squad  G_A_per_90  Key_Passes_per_90  \
4       Cody Gakpo  Liverpool        1.00               2.00   
7   Mohammed Kudus  Tottenham        0.69               2.41   
8    Mohamed Salah  Liverpool        0.67               1.00   
9  Estêvão Willian    Chelsea        0.50               1.00   

   Ball_Recoveries_per_90  
4                    4.00  
7                    3.79  
8                    4.00  
9                    3.00  

🎯 Complete Midfielders (Creativity + Passing + Defense):

🛡️ Complete Defenders (Defense + Ball-Playing + Progression):
               Player            Squad  Tackles_Won_per_90  Pass_Accuracy  \
0     Rayan Aït-Nouri  Manchester City                2.73           86.8   
1  Dominik Szoboszlai        Liverpool                2.33        

In [7]:
conn.close()
print("\n🔚 Position-specific technical analysis complete!")


🔚 Position-specific technical analysis complete!
